All sensors

In [33]:
import re
import subprocess

In [34]:
r_temp = re.compile('t=([0-9]+)')

In [35]:
dct_sensors = {'salon': '28-0000067c14c7',
               'cour': '28-0000050b221b',
               'grenier': '28-15dccf1964ff'}

In [36]:
bus_path = '/home/francis/sys-bus-w1-devices/'

In [37]:
address = bus_path + dct_sensors['salon'] + '/w1_slave'
stdout = subprocess.run(['cat', address], capture_output=True, text=True).stdout

In [38]:
for key, value in dct_sensors.items():

    address = '/home/francis/sys-bus-w1-devices/' + value + '/w1_slave'
    stdout = subprocess.run(['cat', address], capture_output=True, text=True).stdout
    #print (stdout)

    sens_out = r_temp.findall(stdout)
    if len(sens_out) > 0:
        if sens_out[0].isdigit():
            print ('Temp.', key, ':', int(sens_out[0])/1000)


Temp. salon : 19.812
Temp. cour : 1.437
Temp. grenier : 0.75


In [39]:
def read_temperatures ():

        lst_temp = []
        r_temp = re.compile('t=([0-9]+)')
        dct_sensors = {'salon': '28-0000067c14c7',
                        'cour': '28-0000050b221b',
                        'grenier': '28-15dccf1964ff',
                        'cellier': '28-6cb8cf1964ff',
                        'chaufferie': '28-1806ce1964ff'}
        bus_path = '/home/francis/sys-bus-w1-devices/'

        address = bus_path + dct_sensors['salon'] + '/w1_slave'
        stdout = subprocess.run(['cat', address], capture_output=True, text=True).stdout

        for key, value in dct_sensors.items():

                address = '/home/francis/sys-bus-w1-devices/' + value + '/w1_slave'
                stdout = subprocess.run(['cat', address], capture_output=True, text=True).stdout

                sens_out = r_temp.findall(stdout)
                if len(sens_out) > 0:
                        if sens_out[0].isdigit():
                                lst_temp.append(int(sens_out[0])/1000)
                        else:
                                lst_temp.append(None)
                else:
                        lst_temp.append(None)

        return dct_sensors, lst_temp

In [40]:
db_sensors, db_temp = read_temperatures()

In [41]:
print (db_sensors)
print (db_temp)

{'salon': '28-0000067c14c7', 'cour': '28-0000050b221b', 'grenier': '28-15dccf1964ff', 'cellier': '28-6cb8cf1964ff', 'chaufferie': '28-1806ce1964ff'}
[19.812, 1.437, 0.75, None, None]


Fuel (GPIO)

In [42]:
def read_gpio(gpio_no=17):

    gpio_value = None
    r_gpio = re.compile('([0-9])\n')
    gpio_path = '/home/francis/sys-class-gpio-gpio' + str(gpio_no) + '/'

    address = gpio_path + '/value'
    stdout = subprocess.run(['cat', address], capture_output=True, text=True).stdout
    
    gpio_out = r_gpio.findall(stdout)

    if len(gpio_out) > 0:
        if gpio_out[0].isdigit():
                gpio_value = int(gpio_out[0])

    return gpio_value

In [43]:
db_gpio = [read_gpio(17)]
print (db_gpio)

[1]


Test writing the database

In [44]:
import sqlite3
import datetime

In [45]:
db_file = '../database/logdata-test.db'
timestamp = datetime.datetime.now().isoformat()

In [46]:
db_temp.insert(0, timestamp)
db_gpio.insert(0, timestamp)

In [47]:
conn = sqlite3.connect(db_file, timeout=60)
conn.execute("PRAGMA journal_mode=WAL;")
c = conn.cursor()

In [48]:
sql = 'CREATE TABLE IF NOT EXISTS temperatures' + \
        ' (timestamp DATETIME,' + \
        ' temp_1 REAL, temp_2 REAL, temp_3 REAL, temp_4 REAL, temp_5 REAL);'
c.execute(sql)

In [49]:
sql = 'INSERT INTO temperatures (timestamp, temp_1, temp_2, temp_3, temp_4, temp_5)' + \
        ' VALUES (?, ?, ?, ?, ?, ?);'
c.execute(sql, db_temp)

In [50]:
sql = 'CREATE TABLE IF NOT EXISTS gpio' + \
        ' (timestamp DATETIME, gpio_value INT);'
c.execute(sql)

In [51]:
sql = 'INSERT INTO gpio (timestamp, gpio_value)' + \
        ' VALUES (?, ?);'
c.execute(sql, db_gpio)

In [52]:
conn.commit()                
conn.close()